# PayNet assesment

#### Prepared by Ainur Afifah

todo:
1. Merge structure (from nested into single)
2. Separate person name into first and last
3. Convert timestamps into UTC +8
4. Create charts & visualization
5. PII approach with reasons

## Read JSON data

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("cc-data-cleaning-visualization").getOrCreate()
json_file_path = "/Users/ainurafifah/Desktop/PROJECTS/portfolio/PayNet/cc_sample_transaction.json"


try:
    df = spark.read.option("multiline", "true").json(json_file_path)
    df = spark.read.json(json_file_path)
    
    print("Full schema:")
    df.printSchema()
    
    df.show(5, truncate=False, vertical=True)
    
    
except Exception as e:
    print(f"Error reading JSON file: {e}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/22 14:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Full schema:
root
 |-- Unnamed: 0: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- category: string (nullable = true)
 |-- cc_bic: string (nullable = true)
 |-- cc_num: string (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- merch_eff_time: string (nullable = true)
 |-- merch_last_update_time: string (nullable = true)
 |-- merch_lat: string (nullable = true)
 |-- merch_long: string (nullable = true)
 |-- merch_zipcode: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- personal_detail: string (nullable = true)
 |-- trans_date_trans_time: string (nullable = true)
 |-- trans_num: string (nullable = true)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Unnamed: 0             | 0       

## Required changes

### 1. Convert timestamp format

In [2]:
from pyspark.sql.functions import date_format, from_unixtime, to_timestamp, from_utc_timestamp, col

# Convert timestamp to UTC+8
# Set timezone configuration
spark.conf.set("spark.sql.session.timeZone", "UTC+8")
    
df = df.withColumn("trans_date_trans_time_utc8",to_timestamp(col("trans_date_trans_time")))
    
df = df.withColumn("merch_eff_time_utc8",from_unixtime(col("merch_eff_time") / 1000000))

df = df.withColumn("merch_last_update_time_utc8",from_unixtime(col("merch_last_update_time") / 1000))



In [3]:
#Updated schema (3 new timestamp utc8 columns are added)
df.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- category: string (nullable = true)
 |-- cc_bic: string (nullable = true)
 |-- cc_num: string (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- merch_eff_time: string (nullable = true)
 |-- merch_last_update_time: string (nullable = true)
 |-- merch_lat: string (nullable = true)
 |-- merch_long: string (nullable = true)
 |-- merch_zipcode: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- personal_detail: string (nullable = true)
 |-- trans_date_trans_time: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- trans_date_trans_time_utc8: timestamp (nullable = true)
 |-- merch_eff_time_utc8: string (nullable = true)
 |-- merch_last_update_time_utc8: string (nullable = true)



### 2. Flatten the JSON

These 2 columns will appear in the root schema
1. personal_detail
2. address

In [4]:
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.functions import col, from_json, split, concat, lit, substring, regexp_extract, abs
from pyspark.sql.functions import regexp_replace, when, array_contains, size, get_json_object
from pyspark.sql.functions import to_date, year, month, dayofmonth


# Define schema for nested structures
address_schema = StructType([
                        StructField("street", StringType()),
                        StructField("city", StringType()),
                        StructField("state", StringType()),
                        StructField("zip", StringType())
                        ])

personal_detail_schema = StructType([
                        StructField("person_name", StringType()),
                        StructField("gender", StringType()),
                        StructField("address", StringType()),
                        StructField("lat", StringType()),
                        StructField("long", StringType()),
                        StructField("city_pop", StringType()),
                        StructField("job", StringType()),   
                        StructField("dob", StringType())
                        ])


# Parse nested JSON strings
df_parsed = df.withColumn("personal_detail_parsed", from_json("personal_detail", personal_detail_schema))

In [6]:
df_parsed.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- category: string (nullable = true)
 |-- cc_bic: string (nullable = true)
 |-- cc_num: string (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- merch_eff_time: string (nullable = true)
 |-- merch_last_update_time: string (nullable = true)
 |-- merch_lat: string (nullable = true)
 |-- merch_long: string (nullable = true)
 |-- merch_zipcode: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- personal_detail: string (nullable = true)
 |-- trans_date_trans_time: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- trans_date_trans_time_utc8: timestamp (nullable = true)
 |-- merch_eff_time_utc8: string (nullable = true)
 |-- merch_last_update_time_utc8: string (nullable = true)
 |-- personal_detail_parsed: struct (nullable = true)
 |    |-- person_name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- address: string (nullable =

In [7]:
df_final = df_parsed.withColumn("address_parsed",from_json(
                regexp_replace(col("personal_detail_parsed.address"), r'\\"', '"'),
                address_schema))

In [8]:
df_final.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- category: string (nullable = true)
 |-- cc_bic: string (nullable = true)
 |-- cc_num: string (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- merch_eff_time: string (nullable = true)
 |-- merch_last_update_time: string (nullable = true)
 |-- merch_lat: string (nullable = true)
 |-- merch_long: string (nullable = true)
 |-- merch_zipcode: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- personal_detail: string (nullable = true)
 |-- trans_date_trans_time: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- trans_date_trans_time_utc8: timestamp (nullable = true)
 |-- merch_eff_time_utc8: string (nullable = true)
 |-- merch_last_update_time_utc8: string (nullable = true)
 |-- personal_detail_parsed: struct (nullable = true)
 |    |-- person_name: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- address: string (nullable =

### 3. Split the person_name into first & last
1. The name has different type of delimiters
2. First name is 'first' and the second name is 'last'

### 4. Create a new column 'year' from 'dob' 
1. To assess if there is a pattern in age group & fraud cases

### 5. Create new columns from 'merchant'
1. There is an existing is_fraud column. Will require to do a distribution analysis for this column.
2. Bcs merchant column has a 'fraud_' marker in the name. Assumption is the merchant is a fraud case. 
3. Will create 2 columns: fraud_flag & merchant_name. 
4. Original merchant column is still part of the dataframe until further investigation.

In [9]:
import re
# Handling person_name column split
DELIMITERS = [",", "@", "/", "!", "\\\\"]
delimiter_pattern = "|".join([re.escape(d) for d in DELIMITERS])
    
df_final = df_final.withColumn("normalized_name",
        regexp_replace(col("personal_detail_parsed.person_name"), 
                       delimiter_pattern, "|")).withColumn("name_parts",
                                                           split(col("normalized_name"), "\\|"))
    
spark.conf.set("spark.sql.debug.maxToStringFields", "1000")

In [17]:
from pyspark.sql.functions import col, from_json, split, concat, lit, substring, regexp_extract, abs

df_final = df_final.withColumn("fraud_flag",
       regexp_extract(col("merchant"), "^fraud_", 0))

df_final = df_final.withColumn("merchant_name",
       regexp_extract(col("merchant"), "[^_]+$", 0))

#create a Boolean column from fraud_flag 
df_final = df_final.withColumn("derived_fraud_flag",
    when(col("merchant").startswith("fraud_"), True).otherwise(False))

#cast is_fraud column into Boolean type
df_final = df_final.withColumn("is_fraud_bool",
       col("is_fraud").cast("boolean"))


In [18]:
df_final = df_final.withColumn("amount_clean",
       abs(col("amt").cast("decimal(10,2)"))).filter(col("amount_clean") < 100000)

In [19]:
flattened_df = df_final.select(
            "Unnamed: 0",
            "amt",
            "amount_clean",
            "category",
            "cc_bic",
            "cc_num",
            "merch_eff_time",
            "merch_last_update_time",
            "merch_lat",
            "merch_long",
            "merch_zipcode",
            "merchant",
            "merchant_name",
            "fraud_flag",
            "derived_fraud_flag",
            "is_fraud_bool",
            "is_fraud",
            "trans_date_trans_time",
            "trans_num",
            "trans_date_trans_time_utc8",
            "merch_eff_time_utc8",
            "merch_last_update_time_utc8",
            col("personal_detail_parsed.person_name").alias("person_name"),
            
            when((size(col("name_parts")) > 0) & (col("name_parts")[0] != ""), 
            col("name_parts")[0]).otherwise(lit(None)).alias("first"),
        
            when((size(col("name_parts")) > 1) & (col("name_parts")[1] != ""),
            col("name_parts")[1]).otherwise(lit(None)).alias("last"),
    
            col("personal_detail_parsed.gender").alias("gender"),
            col("address_parsed.street").alias("street"),
            col("address_parsed.city").alias("city"),
            col("address_parsed.state").alias("state"),
            col("address_parsed.zip").alias("zip"),
            col("personal_detail_parsed.lat").alias("lat"),
            col("personal_detail_parsed.long").alias("long"),
            col("personal_detail_parsed.city_pop").alias("city_pop"),
            col("personal_detail_parsed.job").alias("job"),
            year(to_date(col("personal_detail_parsed.dob"))).alias("birth_year"),
            col("personal_detail_parsed.dob").alias("dob")   
        )

flattened_df.printSchema()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- amount_clean: decimal(10,2) (nullable = true)
 |-- category: string (nullable = true)
 |-- cc_bic: string (nullable = true)
 |-- cc_num: string (nullable = true)
 |-- merch_eff_time: string (nullable = true)
 |-- merch_last_update_time: string (nullable = true)
 |-- merch_lat: string (nullable = true)
 |-- merch_long: string (nullable = true)
 |-- merch_zipcode: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- fraud_flag: string (nullable = true)
 |-- derived_fraud_flag: boolean (nullable = false)
 |-- is_fraud_bool: boolean (nullable = true)
 |-- is_fraud: string (nullable = true)
 |-- trans_date_trans_time: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- trans_date_trans_time_utc8: timestamp (nullable = true)
 |-- merch_eff_time_utc8: string (nullable = true)
 |-- merch_last_update_time_utc8: string (nullable =

### Summary

In [20]:
flattened_df.show(vertical=True)

-RECORD 0-------------------------------------------
 Unnamed: 0                  | 0                    
 amt                         | 4.97                 
 amount_clean                | 4.97                 
 category                    | misc_net             
 cc_bic                      | CITIUS33CHI          
 cc_num                      | 2703186189652095     
 merch_eff_time              | 1325376018798532     
 merch_last_update_time      | 1325376018666        
 merch_lat                   | 36.011293            
 merch_long                  | -82.048315           
 merch_zipcode               | 28705                
 merchant                    | fraud_Rippin, Kub... 
 merchant_name               | Rippin, Kub and Mann 
 fraud_flag                  | fraud_               
 derived_fraud_flag          | true                 
 is_fraud_bool               | false                
 is_fraud                    | 0                    
 trans_date_trans_time       | 2019-01-01 00:0

In [22]:
# Basic Stats
flattened_df.describe().show(vertical=True)

-RECORD 0-------------------------------------------
 summary                     | count                
 Unnamed: 0                  | 1296675              
 amt                         | 1296675              
 amount_clean                | 1296675              
 category                    | 1296675              
 cc_bic                      | 1296675              
 cc_num                      | 1296675              
 merch_eff_time              | 1296675              
 merch_last_update_time      | 1296675              
 merch_lat                   | 1296675              
 merch_long                  | 1296675              
 merch_zipcode               | 1100702              
 merchant                    | 1296675              
 merchant_name               | 1296675              
 fraud_flag                  | 1296675              
 is_fraud                    | 1296675              
 trans_date_trans_time       | 1296675              
 trans_num                   | 1296675        

#### From the basic statistics summary:
1. Count: merch_zipcode is the only column that does not have the same count number (if not necessary, we can drop this column)
2. min & max :
    1. Column amt has min -> 1.0 and max ->9999.39 with mean-> 70.35 (possible normalization required for this column. Check the median amt) 
    2. Column year (birth) range between 1924 to 2005. (we can check the median year)

In [23]:
flattened_df.summary("count", "min", "max", "25%", "50%", "75%").show(vertical=True)

-RECORD 0-------------------------------------------
 summary                     | count                
 Unnamed: 0                  | 1296675              
 amt                         | 1296675              
 amount_clean                | 1296675              
 category                    | 1296675              
 cc_bic                      | 1296675              
 cc_num                      | 1296675              
 merch_eff_time              | 1296675              
 merch_last_update_time      | 1296675              
 merch_lat                   | 1296675              
 merch_long                  | 1296675              
 merch_zipcode               | 1100702              
 merchant                    | 1296675              
 merchant_name               | 1296675              
 fraud_flag                  | 1296675              
 is_fraud                    | 1296675              
 trans_date_trans_time       | 1296675              
 trans_num                   | 1296675        

#### From the interquartile statistics summary:

1. Column amt: We can see the Q1-Q3 is more close with each other contrary to the max value
2. (Birth) year range column: In the Q1-Q3, we can see that the values are 1962, 1975 and 1987. Which are all under 1990. There is a possible correlation between customer age and fraud cases.

3. merchant name column will be required to have separate fraud_flag column based on their name. Next, we can compare with the existing is_fraud column and how much are the count different or similar from each other.


In [22]:
from pyspark.sql.functions import countDistinct

# Get distinct counts for categorical column
for column in flattened_df.columns:
    flattened_df.select(countDistinct(col(column)).alias(f"distinct_{column}")).show()

+-------------------+
|distinct_Unnamed: 0|
+-------------------+
|            1296675|
+-------------------+



+------------+
|distinct_amt|
+------------+
|       52928|
+------------+



+-----------------+
|distinct_category|
+-----------------+
|               14|
+-----------------+



+---------------+
|distinct_cc_bic|
+---------------+
|              9|
+---------------+



+---------------+
|distinct_cc_num|
+---------------+
|            983|
+---------------+



...

+------------+
|distinct_lat|
+------------+
|         968|
+------------+



+-------------+
|distinct_long|
+-------------+
|          969|
+-------------+



+-----------------+
|distinct_city_pop|
+-----------------+
|              879|
+-----------------+



+------------+
|distinct_job|
+------------+
|         494|
+------------+



+------------+
|distinct_dob|
+------------+
|         968|
+------------+



In [13]:
try:
    # Get distinct category for each categorical column
    selected_cols = ["category", "is_fraud", "merchant", 
                     "gender", "person_name", 
                     "city", 
                     "state", "job", 
                     "city_pop", "dob"]

    for column in selected_cols:
        flattened_df.select(column).distinct().show()

except Exception as e:
    print(f"Error {e}")

+--------------+
|      category|
+--------------+
|        travel|
|      misc_net|
|   grocery_pos|
|     kids_pets|
|  shopping_net|
|   grocery_net|
|   food_dining|
| gas_transport|
| personal_care|
|health_fitness|
| entertainment|
|          home|
|      misc_pos|
|  shopping_pos|
+--------------+



+--------+
|is_fraud|
+--------+
|       0|
|       1|
+--------+



+--------------------+
|            merchant|
+--------------------+
|  fraud_Rau and Sons|
|    fraud_Herman Inc|
|     fraud_Thiel PLC|
|fraud_O'Hara-Wild...|
|fraud_Bradtke, To...|
|fraud_Altenwerth,...|
|fraud_Robel, Cumm...|
| fraud_Greenholt Ltd|
|fraud_Ledner, Har...|
|  fraud_Waelchi-Wolf|
|   fraud_Effertz LLC|
|fraud_Smitham-Sch...|
| fraud_Kihn-Schuster|
|   fraud_Hills-Boyer|
|fraud_Douglas, Du...|
|      fraud_Jast Ltd|
|fraud_Stroman, Hu...|
|fraud_Kerluke-Abs...|
|fraud_Gottlieb-Ha...|
|fraud_Rippin-VonR...|
+--------------------+
only showing top 20 rows



+------+
|gender|
+------+
|     F|
|     M|
+------+



+-----------------+
|             city|
+-----------------+
|Kingsford Heights|
|             Jelm|
|            Tyler|
|          Palermo|
|      Springfield|
|           Corona|
|          Whigham|
|          Tyaskin|
|      Woods Cross|
|           Auburn|
|      New Memphis|
|  North Las Vegas|
|       Plainfield|
|          Acworth|
|          Phoenix|
|            Rhame|
|         Falconer|
|       Georgetown|
|         Doe Hill|
|          Harwood|
+-----------------+
only showing top 20 rows



+-----+
|state|
+-----+
|   AZ|
|   SC|
|   LA|
|   MN|
|   NJ|
|   DC|
|   OR|
|   VA|
|   RI|
|   WY|
|   KY|
|   NH|
|   MI|
|   NV|
|   WI|
|   ID|
|   CA|
|   NE|
|   CT|
|   MT|
+-----+
only showing top 20 rows



+--------------------+
|                 job|
+--------------------+
| Retail merchandiser|
| Librarian, academic|
|Designer, ceramic...|
|Engineer, aeronau...|
|    Catering manager|
|English as a seco...|
|Primary school te...|
| Early years teacher|
|Occupational hygi...|
|Control and instr...|
|   Transport planner|
|Applications deve...|
|       Art therapist|
|  Petroleum engineer|
|       Stage manager|
|Investment banker...|
|Conservator, furn...|
|          Counsellor|
|     Product manager|
|Sports administrator|
+--------------------+
only showing top 20 rows



+--------+
|city_pop|
+--------+
|     467|
|    1512|
|    9993|
|   15269|
|    5657|
|     800|
|   27020|
|  817312|
|   95015|
|   35299|
|      51|
|  841711|
|     475|
|    1897|
|     307|
|     718|
|  233060|
|  601723|
|     205|
|     581|
+--------+
only showing top 20 rows



In [14]:
from pyspark.sql.functions import col, sum

# Count nulls per column
null_counts = flattened_df.select(
    [sum(col(c).isNull().cast("int")).alias(c) for c in flattened_df.columns]
)

null_counts.show(vertical=True)

-RECORD 0-----------------------------
 Unnamed: 0                  | 0      
 amt                         | 0      
 category                    | 0      
 cc_bic                      | 0      
 cc_num                      | 0      
 is_fraud                    | 0      
 merch_eff_time              | 0      
 merch_last_update_time      | 0      
 merch_lat                   | 0      
 merch_long                  | 0      
 merch_zipcode               | 195973 
 merchant                    | 0      
 trans_date_trans_time       | 0      
 trans_num                   | 0      
 trans_date_trans_time_utc8  | 0      
 merch_eff_time_utc8         | 0      
 merch_last_update_time_utc8 | 0      
 first                       | 0      
 last                        | 0      
 gender                      | 0      
 street                      | 0      
 city                        | 0      
 state                       | 0      
 zip                         | 0      
 lat                     

In [15]:
#drop merch_zipcode column bcs of high null counts. other columns has zero null count

flattened_df = flattened_df.drop("merch_zipcode")

flattened_df.show(3, vertical=True)

-RECORD 0-------------------------------------------
 Unnamed: 0                  | 0                    
 amt                         | 4.97                 
 category                    | misc_net             
 cc_bic                      | CITIUS33CHI          
 cc_num                      | 2703186189652095     
 is_fraud                    | 0                    
 merch_eff_time              | 1325376018798532     
 merch_last_update_time      | 1325376018666        
 merch_lat                   | 36.011293            
 merch_long                  | -82.048315           
 merchant                    | fraud_Rippin, Kub... 
 trans_date_trans_time       | 2019-01-01 00:00:18  
 trans_num                   | 0b242abb623afc578... 
 trans_date_trans_time_utc8  | 2019-01-01 00:00:18  
 merch_eff_time_utc8         | 2012-01-01 08:00:18  
 merch_last_update_time_utc8 | 2012-01-01 08:00:18  
 first                       | Jennifer             
 last                        | Banks          